

## Created The Clusters of Iris Flower  



**STEPS**
 - Reading and Understanding the Data
 - Data Preparation
 - Finding the optimal number of cluster
 - cluster profiling
                                                                                                           

## libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

## Reading The Data

In [ ]:
iris=pd.read_csv('../input/iris-flower-dataset/IRIS.csv')
iris.head()
#supervised learning data

In [ ]:
iris.shape

In [ ]:
iris.info()

In [ ]:
iris.isnull().sum()

In [ ]:
iris.describe()

In [ ]:
iris['species'].value_counts()

In [ ]:
# Distplot of all columns
plt.figure(figsize=(8,8))
feature=iris.columns[1:-1]
for i in enumerate(feature):
    plt.subplot(2,2,i[0]+1)
    sns.distplot(iris[i[1]])

In [ ]:
# Boxplot of all columns
plt.figure(figsize=(8,8))
feature=iris.columns[1:-1]
for i in enumerate(feature):
    plt.subplot(2,2,i[0]+1)
    sns.boxplot(iris[i[1]])

**SepalWidthhas outliers so we treated the outliers**

In [ ]:
Q1=iris.sepal_width.quantile(.01)
Q3=iris.sepal_width.quantile(.99)
IQR=Q3-Q1
iris=iris[(iris.sepal_width>= Q1-1.5*IQR) & (iris.sepal_width<= Q3+1.5*IQR)]

In [ ]:
#scaling
scaler=StandardScaler()
iris_scaled=scaler.fit_transform(iris.drop(['species'],axis=1))

In [ ]:
iris_scaled.shape

In [ ]:
#coverting numpy into dataframe
iris_scaled= pd.DataFrame(iris_scaled)
iris_scaled.head()

In [ ]:
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) 
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
 
    return H

In [ ]:
hopkins(iris_scaled)

# **k-means clustering**

In [ ]:
#elbow curve
ssd=[]
range_clusters=[2,3,4,5,6,7,8]

for num_clusters in range_clusters:
    kmeans=KMeans(n_clusters=num_clusters,max_iter=50)
    kmeans.fit(iris_scaled)
    ssd.append(kmeans.inertia_)
    
plt.plot(ssd)

In [ ]:
#silhoue score
range_clusters=[2,3,4,5,6,7,8]

for num_clusters in range_clusters:
    
    #intialise kmeans
    kmeans=KMeans(n_clusters=num_clusters,max_iter=50)
    kmeans.fit(iris_scaled)
    
   
    cluster_labels=kmeans.labels_
    
     #silhouette score
    silhouette_avg=silhouette_score(iris_scaled,cluster_labels)
    print("For n_clusters={0}, the silhouette score is{1}".format(num_clusters,silhouette_avg))

In [ ]:
#final model K=3
kmeans=KMeans(n_clusters=3,max_iter=50)
kmeans.fit(iris_scaled)

In [ ]:
kmeans.labels_

In [ ]:
#assign cluster
iris['cluster_id']=kmeans.labels_
iris.head()

In [ ]:
sns.scatterplot(x='petal_length',y='petal_width',data=iris,hue='cluster_id')
plt.show()

# **hierarchical clustering**

In [ ]:
mergings=linkage(iris_scaled,method='complete',metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
cluster_labels=cut_tree(mergings,n_clusters=3).reshape(-1,)
cluster_labels.shape

In [ ]:
iris['cluster_labels']=cluster_labels
iris

# cluster profiling

In [ ]:
sns.boxplot(x='cluster_labels',y='petal_length',data=iris)
plt.show()

In [ ]:
sns.boxplot(x='cluster_labels',y='petal_width',data=iris)
plt.show()

**THREE CLUSTERS ARE FORMED FOR THREE IRIS FLOWERS-SETOSA,VERSICOLOR,VIRGINICA**

In [ ]:
iris['cluster_id'].value_counts()

In [ ]:
iris['cluster_labels'].value_counts()

**we can see that K-means clustering performing approximately same clustering labels**